<a href="https://colab.research.google.com/github/maym5/lstm_vs_transformer/blob/main/lstm_vs__transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Evaluation Metrics:

For prediction performance: Mean Absolute Percentage Error (MAPE).
Specific implementation here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html

### Import Statements

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import layers
import time
import timeit

## 1. Read Data

In [5]:
amzn = pd.read_pickle("/Raw Data/pickle/amzn_prices_labels_news.pkl")
aapl = pd.read_pickle("/Raw Data/pickle/aapl_prices_labels_news.pkl")
msft = pd.read_pickle("/Raw Data/pickle/msft_prices_labels_news.pkl")

In [92]:
datasets = ['aapl', 'amzn', 'msft']

In [9]:
X_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'neg', 'neu', 'pos', 'compound', 'subjectivity', 'polarity']
y_col = ['Adj Close Next']

In [93]:
def create_train_test_split(company):
  if company == 'amzn':
    X = amzn[X_cols]
    y = amzn[y_col]
  elif company == 'aapl':
    X = aapl[X_cols]
    y = aapl[y_col]
  elif company == 'msft':
    X = msft[X_cols]
    y = msft[y_col]
  split = int(0.8*X.shape[0])
  X_train = X[:split]
  X_test = X[split:]
  y_train = y[:split]
  y_test = y[split:]
  return X_train, X_test, y_train, y_test


In [25]:
def transform(train, test, timestep, num_features) -> np.array:
        train_remainder = train.shape[0] % timestep
        test_remainder = test.shape[0] % timestep
        if train_remainder != 0 and test_remainder != 0:
            train = train[train_remainder:]
            test = test[test_remainder:]
        elif train_remainder != 0:
            train = train[train_remainder:]
        elif test_remainder != 0:
            test = test[test_remainder:]
        return window_and_reshape(train, timestep, num_features), window_and_reshape(test, timestep, num_features)

In [26]:
def window_and_reshape(data, timestep, num_features) -> np.array:
        """
        Reformats data into shape our model needs,
        namely, [# samples, timestep, # feautures]
        samples
        """
        samples = int(data.shape[0] / timestep)
        result = np.array(np.array_split(data, samples))
        return result.reshape((samples, timestep, num_features))

In [83]:
def build_lstm(X_train, X_test, y_train, y_test, epochs=25, batch_size=32) -> tf.keras.Model:
  """
  Builds, compiles, and fits our LSTM baseline model.
  """
  n_timesteps, n_features, n_outputs = 5, 12, 5
  callbacks = [tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
  model = Sequential()
  model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(n_outputs))
  print('compiling baseline model...')
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])
  print('fitting model...')
  start = time.time()
  history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1, callbacks=callbacks)
  print(time.time() - start)
  return model, history

In [94]:
for company in datasets:
  print("Training and Evaluating ",company, " data: ")
  X_train, X_test, y_train, y_test = create_train_test_split(company)
  X_train_transform, X_test_transform = transform(X_train, X_test, 5, 12)
  y_train_transform, y_test_transform = transform(y_train, y_test, 5, 1)
  baseline = build_lstm(X_train_transform, X_test_transform, y_train_transform, y_test_transform)
  model = baseline[0]
  history = baseline[1]
  print(model.summary())
  print("Model Results: ")
  print(model.evaluate(X_test_transform, y_test_transform))



Training and Evaluating  aapl  data: 
compiling baseline model...
fitting model...
Epoch 1/25
2/2 [==============================] - 2s 309ms/step - loss: 99855959588864.0000 - mae: 7619821.5000 - mape: 4838167.0000 - val_loss: 23357781180416.0000 - val_mae: 3501547.2500 - val_mape: 1910328.0000
Epoch 2/25
2/2 [==============================] - 0s 45ms/step - loss: 44345495388160.0000 - mae: 4794349.0000 - mape: 3046333.5000 - val_loss: 11546132480000.0000 - val_mae: 2420101.2500 - val_mape: 1319708.2500
Epoch 3/25
2/2 [==============================] - 0s 49ms/step - loss: 21170319523840.0000 - mae: 3282619.7500 - mape: 2086445.1250 - val_loss: 4942861959168.0000 - val_mae: 1666818.7500 - val_mape: 907233.5000
Epoch 4/25
2/2 [==============================] - 0s 49ms/step - loss: 9314932621312.0000 - mae: 2186402.0000 - mape: 1386251.0000 - val_loss: 2310930694144.0000 - val_mae: 1163051.7500 - val_mape: 630299.1250
Epoch 5/25
2/2 [==============================] - 0s 42ms/step - loss

compiling baseline model...
fitting model...
Epoch 1/25
2/2 [==============================] - 2s 302ms/step - loss: 26880535166976.0000 - mae: 3885040.5000 - mape: 3132946.5000 - val_loss: 7157158248448.0000 - val_mae: 1981251.8750 - val_mape: 1610326.2500
Epoch 2/25
2/2 [==============================] - 0s 46ms/step - loss: 8833703346176.0000 - mae: 2234407.0000 - mape: 1784570.1250 - val_loss: 2208752992256.0000 - val_mae: 1203596.6250 - val_mape: 984545.3750
Epoch 3/25
2/2 [==============================] - 0s 58ms/step - loss: 2784062865408.0000 - mae: 1266885.5000 - mape: 1011836.5625 - val_loss: 1762551922688.0000 - val_mae: 1013561.8750 - val_mape: 824476.6250
Epoch 4/25
2/2 [==============================] - 0s 58ms/step - loss: 1939290324992.0000 - mae: 1090399.1250 - mape: 881119.3125 - val_loss: 1703017185280.0000 - val_mae: 1004983.3125 - val_mape: 815718.0000
Epoch 5/25
2/2 [==============================] - 0s 49ms/step - loss: 2337452589056.0000 - mae: 1176376.5000 - m

compiling baseline model...
fitting model...
Epoch 1/25
2/2 [==============================] - 2s 288ms/step - loss: 3352867635200.0000 - mae: 1513592.0000 - mape: 551666.1250 - val_loss: 1339487944704.0000 - val_mae: 975756.1250 - val_mape: 305894.8750
Epoch 2/25
2/2 [==============================] - 0s 44ms/step - loss: 1301676163072.0000 - mae: 933435.9375 - mape: 339000.5312 - val_loss: 532757413888.0000 - val_mae: 589567.3125 - val_mape: 184019.5156
Epoch 3/25
2/2 [==============================] - 0s 61ms/step - loss: 547250700288.0000 - mae: 592475.5000 - mape: 216635.4375 - val_loss: 224755171328.0000 - val_mae: 381381.7812 - val_mape: 118417.5469
Epoch 4/25
2/2 [==============================] - 0s 62ms/step - loss: 299222564864.0000 - mae: 397534.1250 - mape: 145599.0625 - val_loss: 216264540160.0000 - val_mae: 351866.0000 - val_mape: 109852.2422
Epoch 5/25
2/2 [==============================] - 0s 64ms/step - loss: 323066396672.0000 - mae: 423287.7812 - mape: 154791.9375 - 

In [95]:
def plot_results(test, preds, df, image_path=None, title_suffix=None, xlabel='AAPL stock Price'):
  """
  Plots training data in blue, actual values in red, and predictions in green,
  over time.
  """
  fig, ax = plt.subplots(figsize=(20,6))
  # x = df.Close[-498:].index
  plot_test = test[1:]
  plot_preds = preds[1:]
  x = df[-(plot_test.shape[0]*plot_test.shape[1]):].index
  plot_test = plot_test.reshape((plot_test.shape[0]*plot_test.shape[1], 1))
  plot_preds = plot_preds.reshape((plot_test.shape[0]*plot_test.shape[1], 1))
  ax.plot(x, plot_test, label='actual')
  ax.plot(x, plot_preds, label='preds')
  if title_suffix==None:
    ax.set_title('Predictions vs. Actual')
  else:
    ax.set_title(f'Predictions vs. Actual, {title_suffix}')
  ax.set_xlabel('Date')
  ax.set_ylabel(xlabel)
  ax.legend()
  plt.show()